## Import everything needed

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from sql import get_data
import psycopg2 # Needed to get database errors when uploading dataframe
import sql
import sqlalchemy

import warnings # hides warning messages
warnings.filterwarnings("ignore")
import re

## Get data from SQL-Database

In [2]:
jobs_df = get_data('SELECT * FROM capstone.jobs')

## Assigning keywords to Categories

In [3]:
keywords = { 1 : ['Account', 'Customer Service', 'Customer Support', 'Customer Journey', 'Customer Training'],

             2 : ['student', 'intern', 'graduate', 'trainee', 'students', 'Werkstudent', 'thesis'],

             3 : [r'\bSAP\b', r'\bHCM\b', r'\bIT\b', 'Full Stack', 'Front End', 'Back End', 'Software', 'Data Architect', 'Cloud', 'Frontend', 
                'Backend', 'Front-end', 'Back-end','Artificial Intelligence', 'Machine learning', r'\bML\b', r'\bAI\b', 'Linux', 'Data Engineer', 
                'Digital', 'Security Engineer', 'Cybersecurity', 'Cyber', r'\bSW\b', 'Embedded', r'\bC#\b', '.Net', 'UX/UI', r'\bUX\b', r'\bUI\b', 
                'Skywise', 'Network Security'],

            4 : ['Supply Chain', 'Material', 'Material Service', 'Warehouse', 'Buyer', 'Logistic', 'Store', 'Storekeeper', 'Customs', 'Export', 
                'Buying', 'SUPPLIES AND STOCKS', 'Supply Manager', 'Procurement'],

            5 : ['Employee Relations', r'\bHR\b' , 'Health', 'Welfare', 'Claims', 'Human Resources', 'Recruiter', 'Wellbeing', 
                'Leave Program', 'HR Analytics', 'HR Executive', 'Recruitment', 'People'],

            6 : ['Calibration', 'Quality', 'Quality Engineer', 'Quality Inspector'],

            7 : ['Trainer'],

            8 : ['Data Analyst', 'Data Science', 'Senior Business', 'Strategy', 'Decision Support', 'Strategic', 'Forecast', 'Business Analyst', 
                'Operations Analyst', 'Data Scientist', 'Web Analyst', 'Intelligence'],

            9 : ['Crafter', 'Craftsman', 'Craftswoman', 'Painting', 'Maintenance', 'Technician', 'Cabin Maintenance', 'Line Maintenance', 
                'Base Maintenance', 'Heavy Maintenance', 'Component Maintenance', 'Avionic(s)', r'\bIFE\b', 'In-Flight Entertainment', 'TechOps', 
                'AOG', 'Planner', 'Repair. Sheet Metal', r'\bNDT\b', r'\bCNC\b', 'Machinist', 'Installer', 'Painter', 'Shop Manager', 'Manufacturing', 
                'Saddler', 'Structure', 'B1', 'B2', 'B1/B2', 'welder', 'technical', 'Aircraft mechanic', 'mechanic', 'Machine Shop'],

            10 : ['Invoice', 'Controller', 'Financial', 'Finance', 'Accounting', r'\bTax\b', 'Cost Analyst'],

            11 : ['Propulsion', r'\bEngine\b', r'\bCFM\b', r'\bRR\b', 'Rolly-Royce', r'\bPW\b', 'Engine Mechanic', 'PowerPlant'],

            12 : ['Sales', 'Sales Support', 'Sales Development', 'Global Sales', 'Domestic Sales', 'Corporate Sales', 'Sales Account', 'E-Sales', 'Sales Manager'],

            13 : ['Fuel Cell, Sustainability', 'Green', 'Hydrogen', 'Zero', 'H2', 'LH2', 'H2 Engineer', 'Hybrid', 
                 'Electric Propulsion', 'H2 tank', r'\bSAF\b', 'sustainable Aviation Fuel', 'climate',  'cryogen'],
            
            14 : ['Engineer', 'Engineering', 'F&E', r'\bCAD\b', 'Service Engineer', 'Repair Development', 'Certification Engineer',
                 'Design Engineer', 'Ingénieur', 'Entwicklungsingenieur', 'Ingenieur', 'Test Engineer', 'Weight & Balance', 
                 'Computational', 'Avionics Engineer', 'Mechanical Engineer', 'Electrical Engineer'],
            15 : ['Marketing']
}

## Loop through complete DF to find categories in 'Job-Title'

In [4]:
def find_job_keyword(jobs_df):

    for row in range(len(jobs_df)):
        for key in keywords:
            for value in keywords[key]:
                job_title_low = jobs_df['job_title_en'][row].lower()
                keyword_low = value.lower()

                if re.search(keyword_low, job_title_low):
                    save_job_categories(jobs_df['id'][row], key)

In [ ]:
find_job_keyword(jobs_df)

# Save categories in database:

In [5]:
def save_job_categories(jcat_id, jobs_id):
    connection = psycopg2.connect(user="postgres",
                                    password="]+Dr\DXm()`o=L:+",
                                    host="35.234.81.23",
                                    port="5432",
                                    database="postgres")
    
    try:
        
        cursor = connection.cursor()

        postgres_insert_query = f"INSERT INTO capstone.jobs_jcat (jobs_id, jcat_id) VALUES ({jobs_id}, {jcat_id})"
        print(postgres_insert_query)
        cursor.execute(postgres_insert_query)

        connection.commit()
        count = cursor.rowcount
        print(count, "Record inserted successfully into mobile table")

    except (Exception, psycopg2.Error) as error:
        print("Failed to insert record into mobile table", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")